## using imbalanced-learn samplers API

In [ ]:
import os
from pathlib import Path
import sys

In [ ]:
project_name = 'clpsych'
project_path = Path(os.getcwd()).parent
src_path = Path(project_path, 'src')
utils_path = Path(project_path, 'utils')

if sys.platform == "win32":
    data_path = 'D:\Dataset\{0}\dataset'.format(project_name)
    embedding_path = 'D:\Dataset\{0}\embedding'.format(project_name)
    model_path = 'D:\Dataset\{0}\embedding'.format(project_name)
    
elif sys.platform == 'darwin':
    data_path = '/Volumes/Dataset/{0}/dataset'.format(project_name)
    embedding_path = '/Volumes/Dataset/{0}/embedding'.format(project_name)
    
else:
    data_path = Path(project_path, 'dataset')
    model_path = Path(project_path, 'models')
    embedding_path = Path(project_path, 'embedding')

# including the project folder and the utils folder
if str(utils_path) not in ''.join(sys.path):
    sys.path.extend([str(project_path), str(utils_path), str(src_path)])

print('project path = {0}'.format(project_path))
print('data path = {0}'.format(data_path))
print('model path = {0}'.format(model_path))
print('embedding path = {0}'.format(embedding_path))
print('sys.path = {0}'.format(sys.path))

In [ ]:
# utils
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
from numpy import interp 
from collections import Counter
# evaluation metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
# optimizer
from imblearn.over_sampling import RandomOverSampler
from keras.utils import multi_gpu_model
from sklearn.preprocessing import MinMaxScaler
# ml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# deeplearning
import tensorflow as tf
from keras.layers import Reshape, Flatten, Concatenate
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Dropout
from keras import metrics
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam, Adam
from keras.regularizers import l1_l2
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from keras.layers import Input

%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 13]

# seed for numpy and sklearn
random_state = 7
np.random.seed(random_state)

In [ ]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
assert 'GPU' in str(device_lib.list_local_devices())
print('Tensorflow recognizes GPUs')

# confirm Keras sees the GPU
from keras import backend
available_gpu = backend.tensorflow_backend._get_available_gpus()
assert len(available_gpu) > 0
available_gpus = len(available_gpu)
print('number of available GPUs = {0}'.format(available_gpus))
print('list of GPUs = {0}\n'.format(available_gpu))

In [ ]:
def class_report(y_true, y_pred, y_score=None, average='macro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true {0} is not the same shape as y_pred {1}".format(
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average=average))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred-cnt'] = pred_cnt
    class_report_df['pred-cnt'].iloc[-1] = total

    if not (y_score is None):
        # false positive rate
        fpr = dict()
        # true positive rate
        tpr = dict()
        roc_auc = dict()
        for label_ix, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_ix])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

In [ ]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [ ]:
filename = Path(data_path, 'static_features_pandas_v2.pkl')
filename = Path(data_path, 'static_features_pandas_aug_v2.2.pkl')
dataset = pd.read_pickle(filename)
filename = Path(data_path, 'extracted_features_sentiment_per_user_filtered.csv')
sentiment_statics = pd.read_csv(filename)
sentiment_statics.set_index('user_id', inplace=True)
# sentiment_statics.drop('risk_label', axis=1, inplace=True)
n_columns = ['SETLLL__{0}'.format(x) for x in sentiment_statics.columns]
sentiment_statics.columns = n_columns
print(dataset.shape)
dataset = dataset.merge(right=sentiment_statics, left_index=True, right_index=True)
print(dataset.shape)
dataset.to_pickle(Path(data_path, 'static_features_pandas_aug_v2.2.pkl'))

In [ ]:
filename = 'static_features_pandas_v2.pkl'
filename = 'static_features_pandas_aug_v2.pkl'
filename = 'static_features_pandas_aug_v2.2.pkl'
dataset = pd.read_pickle(Path(data_path, filename))
print(dataset.shape)
dataset.head()

In [ ]:
# define 5-fold cross validation test harness
k_fold = pd.read_csv(Path(data_path, 'clpsych19_public_crossvalidation_splits.csv'), header=None,
                    names=['fold', 'train_text', 'user_id'])

# keep non conrol user ids
k_fold = k_fold[k_fold['user_id'] > 0]

<pre>
Feature Set   Description  
A             static_derieved_features  
B             post_coount_by_subreddit
C             lexicon_count  
D             sentiments_macro  
E             sentiments_micro  
F             empathy
G             readability
H             social_context
I             srl
J             ctakes
K             sentiment statics
L             sentiment statics filtered

In [ ]:
data = dataset.copy()
data.shape

In [ ]:
# features_interest = ['SETAAA'] 
# features_interest = ['SETBBB'] # contains NaN
# features_interest = ['SETCCC'] # contains NaN
# features_interest = ['SETDDD']
# features_interest = ['SETEEE']
# features_interest = ['SETFFF'] # contains NaN
# features_interest = ['SETGGG'] # contains NaN
# features_interest = ['SETHHH'] # contains NaN
# features_interest = ['SETIII'] # contains NaN
# features_interest = ['SETJJJ'] # contains NaN
# features_interest = ['SETKKK'] # contains NaN
# features_interest = ['SETLLL'] # contains NaN
# features_interest = ['SETAAA', 'SETDDD', 'SETEEE', 'SETFFF', 'SETGGG']
features_interest = ['SETAAA', 'SETDDD', 'SETEEE']

features = [x for x in dataset.columns if x.split('__')[0] in features_interest]
# including label
features.append('target')
data = dataset[features].copy()
data.shape

In [ ]:
classifier = 'nn2'

ave_f1_scores = list()

print('5 fold CV starting')
for fold_ix in range(1,6):
    
    print('\nFold = {0}'.format(fold_ix))
    
    train_ix = k_fold[(k_fold['fold'] == fold_ix) & (k_fold['train_text'] == 'training')]['user_id']
    test_ix = k_fold[(k_fold['fold'] == fold_ix) & (k_fold['train_text'] == 'test')]['user_id']
    
    x_train = data[data.index.isin(train_ix)].copy()
    x_test = data[data.index.isin(test_ix)].copy()
    
    x_train = x_train.fillna(0)
    x_test = x_test.fillna(0)
    
    y_train = x_train['target']
    x_train.drop(['target'], axis=1, inplace=True)
    
    y_test = x_test['target']
    x_test.drop(['target'], axis=1, inplace=True)
    
    print(x_train.shape)
    print(sorted(Counter(y_train).items()))
    ros = RandomOverSampler(random_state=0)
    x_train, y_train = ros.fit_resample(x_train, y_train)
    print(sorted(Counter(y_train).items()))
    
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    
    # one-hot encoding
    n_y_train = pd.get_dummies(y_train)
    n_y_test = pd.get_dummies(y_test)
    
    if classifier == 'nn':
        
        model = Sequential()
        model.add(Dense(30, input_dim=np.shape(x_train)[1], activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(4, activation='softmax'))
        
        adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        model = multi_gpu_model(model, gpus=available_gpus)
        model.compile(optimizer=adam, loss=f1_loss,
                      metrics=[f1])
        model.fit(x_train, n_y_train, epochs=100, batch_size=100, verbose=1)
        y_pred_train = np.argmax(model.predict(x_train), axis=1) + 1
        y_pred_test = np.argmax(model.predict(x_test), axis=1) + 1
        y_score = model.predict(x_test)
        
    elif classifier == 'nn2':
        
        # divide training dataset
        ix_2_4 = (y_train == 2) | (y_train == 4)
        ix_1_3 = (y_train == 1) | (y_train == 3)

        # fetch the right data points
        x_train_24 = x_train[ix_2_4]
        y_train_24 = y_train[ix_2_4]
        x_train_13 = x_train[ix_1_3]
        y_train_13 = y_train[ix_1_3]
        
        # one-hot encoding
        n_y_train = pd.get_dummies(y_train_13)
        
        model13 = Sequential()
        model13.add(Dense(30, input_dim=np.shape(x_train)[1], activation='relu'))
        model13.add(Dropout(0.2))
        model13.add(Dense(2, activation='softmax'))
        
        adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        model13 = multi_gpu_model(model13, gpus=available_gpus)
        model13.compile(optimizer=adam, loss=f1_loss,
                      metrics=[f1])
        model13.fit(x_train_13, n_y_train, epochs=100, batch_size=100, verbose=1)
        
        # one-hot encoding
        n_y_train = pd.get_dummies(y_train_24)
        
        model24 = Sequential()
        model24.add(Dense(30, input_dim=np.shape(x_train)[1], activation='relu'))
        model24.add(Dropout(0.2))
        model24.add(Dense(2, activation='softmax'))
        
        adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        model24 = multi_gpu_model(model24, gpus=available_gpus)
        model24.compile(optimizer=adam, loss=f1_loss,
                      metrics=[f1])
        model24.fit(x_train_24, n_y_train, epochs=100, batch_size=100, verbose=1)
        
        y_pred_test_13 = model13.predict(x_test)
        y_pred_test_24 = model24.predict(x_test)

        y_score = np.concatenate((y_pred_test_13, y_pred_test_24), axis=1)
        y_pred_test = np.argmax(y_score, axis=1) + 1
        
    elif classifier == 'logit':
        model = LogisticRegression(class_weight='balanced', n_jobs=-1,
                          multi_class='auto', solver='lbfgs',
                          tol=0.00001, C=10, max_iter=1, verbose=True,
                          random_state=random_state)  
        model.fit(x_train, y_train)
        y_pred_test = model.predict(x_test)
        y_score=model.predict_proba(x_test)
    
    elif classifier == 'logit2':
        
        # 1-2,3-4 = 0.263955
        # 1-3,2-4 = 0.332951
        # 1-4,2-3 = 0.193993

        # divide training dataset
        ix_2_4 = (y_train == 2) | (y_train == 4)
        ix_1_3 = (y_train == 1) | (y_train == 3)

        # fetch the right data points
        x_train_24 = x_train[ix_2_4]
        y_train_24 = y_train[ix_2_4]
        x_train_13 = x_train[ix_1_3]
        y_train_13 = y_train[ix_1_3]

        # first model
        model24 = LogisticRegression(class_weight='balanced', n_jobs=-1,
                          multi_class='auto', solver='lbfgs',
                          tol=0.00001, C=10, max_iter=1, verbose=True,
                          random_state=random_state)  
        model24.fit(x_train_24, y_train_24)

        # second model
        model13 = LogisticRegression(class_weight='balanced', n_jobs=-1,
                          multi_class='auto', solver='lbfgs',
                          tol=0.00001, C=10, max_iter=1, verbose=True,
                          random_state=random_state)  
        model13.fit(x_train_13, y_train_13)

        y_pred_test_13 = model13.predict(x_train_13)
        y_score_13 = model13.predict_proba(x_train_13)
        print(class_report( y_true=y_train_13, y_pred=y_pred_test_13, y_score=y_score_13, average='macro'))

        y_pred_test_24 = model24.predict(x_train_24)
        y_score_24 = model24.predict_proba(x_train_24)
        print(class_report( y_true=y_train_24, y_pred=y_pred_test_24, y_score=y_score_24, average='macro'))

        y_pred_test_13 = model13.predict_log_proba(x_test)
        y_pred_test_24 = model24.predict_log_proba(x_test)

        y_score = np.concatenate((y_pred_test_13, y_pred_test_24), axis=1)
        y_pred_test = np.argmax(y_score, axis=1) + 1
    
    report_with_auc = class_report( y_true=y_test, 
                                   y_pred=y_pred_test, 
                                   y_score=y_score,
                                   average='macro')
        
    ave_f1_scores.append(report_with_auc['f1-score'].values[-1])

    cv_column = [fold_ix]
    cv_column.extend( [''] * (report_with_auc.index.shape[0] - 1))
    report_with_auc['Fold'] = cv_column
    report_with_auc['Risk-Factor'] = report_with_auc.index
    report_with_auc = report_with_auc.set_index(['Fold', 'Risk-Factor'])

    if fold_ix == 1:
        report_with_auc_df = report_with_auc.copy()
    else:
        report_with_auc_df = report_with_auc_df.append(report_with_auc.copy())

In [ ]:
report_with_auc_df

In [ ]:
print('average f1-score (all folds) = {0}'.format(np.mean(ave_f1_scores)))

<pre>
'all'    
'SETAAA' 
'SETBBB' 
'SETCCC'
'SETDDD'
'SETEEE'
'SETFFF' 
'SETGGG' 
'SETHHH' 
'SETIII' 
'SETJJJ' 
'SETAAA', 'SETDDD', 'SETEEE', 'SETFFF', 'SETGGG'
'SETAAA', 'SETDDD', 'SETEEE', 'SETEEE'
'SETAAA', 'SETKKK' 0.20175058667560125 (droptout)
'SETAAA', 'SETLLL' 0.12430375888494909 (dropout)
'SETAAA', 'SETDDD', 'SETEEE', 'SETLLL', 'SETHHH', 'SETGGG' 0.11088291908006576 (dropout)
'SETAAA', 'SETDDD', 'SETEEE', 'SETLLL', 'SETHHH', 'SETGGG' 0.07753291443154947 (reg)
'SETAAA', 'SETDDD', 'SETEEE'  0.17349602442078949 (drop)

(nn) (dropout)
'SETAAA', 'SETDDD', 'SETEEE' 0.19014727458966665

(nn) (dropout) (1hl)
'SETAAA', 'SETDDD', 'SETEEE' 0.3255926882010475

(logit)
'SETAAA', 'SETDDD', 'SETEEE'  0.32241407748653905 
'SETAAA', 'SETDDD', 'SETEEE', 'SETLLL' 0.24496511781531444
'SETAAA', 'SETDDD', 'SETEEE', 'SETFFF', 'SETGGG' 0.11353938632270907
'SETAAA', 'SETDDD', 'SETEEE', 'SETFFF' 0.12148581335816777
'SETAAA', 'SETDDD', 'SETEEE', 'SETGGG' 0.25394562653592845